1. How to represent single trace operators?
2. How to implement commutators of these

In [1]:
import numpy as np
import sympy as sp
from typing import Union, Self
from numbers import Number

In [36]:
class SingleTraceOperator():

    def __init__(self, operators: Union[str, list[str]], coeffs: Union[Number, list[Number]], tol=1e-10):
        '''
        TODO
        consider allowing for symbolic coefficients
        consider changing the input to be a dictionary
        build some unit tests to check the basic operation
        Should this be a single trace operator or just operator?
        '''
        if not isinstance(operators, list):
            operators = [operators]
        if not isinstance(coeffs, list):
            coeffs = [coeffs]
        self.tol = tol
        self.data = {op: c for op, c in zip(operators, coeffs) if np.abs(c) > self.tol}
        self.operators = list(self.data.keys())
        self.coeffs = list(self.data.values())
        self._validate()

    def _validate(self):
        if len(self.coeffs) != len(self.operators):
            raise ValueError("Warning, unequal numbers of terms and coefficients were specified.")

    def __repr__(self) -> str:
        return f"SingleTraceOperator(operators={self.operators!r}, coeff={self.coeffs!r})"

    def __str__(self) -> str:
        x = ''
        for idx, (coeff, op) in enumerate(zip(self.coeffs, self.operators)):
            x += f'{coeff}' + f' tr {op}'
            if idx != len(self.operators) - 1:
                x += ' + '
        return x

    def __add__(self, other: Self) -> Self:
        new_data = self.data.copy()
        for op, coeff in other.data.items():
            new_data[op] = new_data.get(op, 0) + coeff
        return SingleTraceOperator(operators=list(new_data.keys()), coeffs=list(new_data.values()))

    def __sub__(self, other: Self) -> Self:
        new_data = self.data.copy()
        for op, coeff in other.data.items():
            new_data[op] = new_data.get(op, 0) - coeff
        return SingleTraceOperator(operators=list(new_data.keys()), coeffs=list(new_data.values()))

    def __rmul__(self, other: Number):
        if isinstance(other, Number):
            new_data = {op: other * coeff for op, coeff in self.data.items()}
            return SingleTraceOperator(operators=list(new_data.keys()), coeffs=list(new_data.values()))
        else:
            raise ValueError("Warning, right multiplication only valid for numbers")

    def __mul__(self, other: Union[Self, Number]):
        if isinstance(other, Number):
            return self.__rmul__(other)
        new_data = {}
        for op1, coeff1 in self.data.items():
            for op2, coeff2 in other.data.items():
                new_data[op1 + op2] = new_data.get(op1 + op2, 0) + coeff1 * coeff2
        return SingleTraceOperator(operators=list(new_data.keys()), coeffs=list(new_data.values()))

    def __pow__(self, power: int):
        if not isinstance(power, int):
            raise ValueError("Warning, exponentiation only defined for positive integer powers.")
        if power < 0:
            raise ValueError("Warning, exponentiation only defined for positive integer powers.")
        if power == 0:
            raise NotImplementedError
        if power == 1:
            return self
        else:
            return self * self.__pow__(power-1)

In [37]:
H = SingleTraceOperator(operators = ['X', 'XY'], coeffs = [0.5, 0])
A = SingleTraceOperator(operators = ['Z', 'XY'], coeffs = [3, 0])
H + A

SingleTraceOperator(operators=['X', 'Z'], coeff=[0.5, 3])

In [38]:
print(H+A)

0.5 tr X + 3 tr Z


In [39]:
H**3

SingleTraceOperator(operators=['XXX'], coeff=[0.125])

In [32]:
H * A

SingleTraceOperator(str_repr=['XZ'], coeff=[1.5])

In [33]:
A * H

SingleTraceOperator(str_repr=['ZX'], coeff=[1.5])

In [34]:
2 * A

SingleTraceOperator(str_repr=['Z'], coeff=[6])